In [2]:
import numpy as np
import functools
from functools import total_ordering
import random 

Written by: George Grattan (Applied Math '22) for Dinesh Mehta for CSCI 406, Algorithms, at Colorado School of Mines.

Note: A good way to think about this problem once you have generated the different rotations for the 4D objects, consider the 3D projections just a 3 dimensional box, and the W dimension as weight. Thus the goal of the problem becomes fitting boxes inside one another so that you can maximize the weight. And once the boxes are sorted by decreasing volume, this becomes an instance of Longest Increasing Subsequence. This interpretation gives an intuitive background to the problem itself, however it can ONLY be applied after all the rotations of the boxes have been made.

Last Edited: 7/27/2021

In [30]:
#define block class
class block:
  #initialize the block
  def __init__(self,w,x,y,z):
    self.W=w
    #The following ensure each block satisfies the constraints of the problem such that x<=y<=z
    #by doing this implicitly in the initialization of a block object I can simplify rotating them later
    projection=[x,y,z]
    projection.sort()
    self.X=projection[0]
    self.Y=projection[1]
    self.Z=projection[2]
  #calculate the volume
    self.V=x*y*z
  #these define <, >, and = so that a list of block objects can be sorted using the sorted() command
  def __lt__(self, obj):
    return ((self.V) < (obj.V))
  def __gt__(self,obj):
    return ((self.V > (obj.V)))
  def __eq__(self,obj):
    return ((self.V)==(obj.V))
  #defines if the 3D projection of the object passed, obj, fits inside of the self
  def fits(self, deez):
    return ((self.X > deez.X) and (self.Y > deez.Y) and (self.Z > deez.Z))

In [4]:
#Input: block object
#Output: A array of 4 block objects, cooresponding to the rotations possible within the problem
def rotate(block_object):
  rotations=[]
  #these are the 4 rotations that the 4D block can make within the constraints of the problem such that x<=y<=z 
  rotations.append(block_object)
  rotations.append(block(block_object.X,block_object.W,block_object.Y,block_object.Z))
  rotations.append(block(block_object.Y,block_object.X,block_object.W,block_object.Z))
  rotations.append(block(block_object.Z,block_object.X,block_object.Y,block_object.W))
  
  return rotations

In [5]:
#Input: Accepts array of n block Objects
#Output:  Array of 4n block objects cooresponding to the 4 rotations the problem allows, sorted by decreasing volume of the 3D projecton V=x*y*z
def make_rotations(blocks):
  #create an empty array
  rotations=[]
  #iterate through the blocks
  for block in blocks:
    #create the 4 rotations for each block and add them to the list of rotations
    rotations+=rotate(block)
  #once you have all the rotations, sort them by volume in decreasing order
  rotations=sorted(rotations,reverse=True)
  #return the list of 4n rotations
  return rotations


In [6]:
def generate_blocks(n):
  blocks=[]
  for i in range(n):
    dim=[]
    for i in range(4):
      #choose a random integer for each dimension
      random_value=random.randint(1,4*n)
      dim.append(random_value)
    #create a block object with the 4 random integers
    blocks.append(block(dim[0],dim[1],dim[2],dim[3]))
  
  return blocks

In [89]:
def print_blocks(blocks):
  print("Volume\t W\tX\tY\tZ")
  print("--------------------------------------------------------")
  for deez in blocks:
    print(deez.V, "\t",deez.W,"\t", deez.X,"\t",deez.Y,"\t", deez.Z)
  print("--------------------------------------------------------\n\n")

In [110]:
#Dynamic Programming Algorithm
#Input: The list of all blocks rotations
#Output: The maximum block stack weight
def DP(blocks):
  #put the weights into the array, which will act as our lookup table
  n=len(blocks)
  msw=[0]*n
  previous=[0]*n
  for i in range(n):
    msw[i]=blocks[i].W
    previous[i]=i

  #iterate through all blocks
  for i in range(1, n):
    for j in range(0, i):
      #if block i fits inside of block j
      if (blocks[j].fits(blocks[i])):
        #if the current maximum stack weight is less than the max stack weight of i inside j, then reset it to i inside j
        if (msw[i] < msw[j] + blocks[i].W):
            msw[i] = msw[j] + blocks[i].W
            #update i with the next best block that it fits within
            previous[i]=j
  maxm = -1
  idx=-1
  #this finds the maximum value of the sum and the index of the final block added to the stack 
  for i in range(n):
    maxm = max(maxm, msw[i])
    if (maxm == msw[i]):
      idx=i
  #sets the starting index to be the value of the last block added
  seq=[blocks[idx]]
  #traces the blocks in the best stack back from the final block to the starting block
  while (idx!=previous[idx]):
    idx=previous[idx]
    seq.append(blocks[idx])
    #returns the max value and the blocks in the stack
  return maxm, reversed(seq)  
  

In [115]:
#Driver Code
blocks=generate_blocks(5)
print("Original 4D Blocks")
print_blocks(blocks)
blocks= make_rotations(blocks)
print("All rotations of those blocks")
print_blocks(blocks)
max_weight, max_stack=DP(blocks)
print("Dynamic Programming Max Weight:", max_weight)
print("Max Stack of Blocks:")
print_blocks(max_stack)

Original 4D Blocks
Volume	 W	X	Y	Z
--------------------------------------------------------
1040 	 12 	 5 	 13 	 16
72 	 17 	 1 	 4 	 18
2394 	 2 	 9 	 14 	 19
918 	 4 	 3 	 17 	 18
234 	 5 	 3 	 6 	 13
--------------------------------------------------------


All rotations of those blocks
Volume	 W	X	Y	Z
--------------------------------------------------------
2496 	 5 	 12 	 13 	 16
2394 	 2 	 9 	 14 	 19
1224 	 1 	 4 	 17 	 18
1224 	 3 	 4 	 17 	 18
1040 	 12 	 5 	 13 	 16
960 	 13 	 5 	 12 	 16
918 	 4 	 3 	 17 	 18
780 	 16 	 5 	 12 	 13
532 	 9 	 2 	 14 	 19
390 	 3 	 5 	 6 	 13
342 	 14 	 2 	 9 	 19
306 	 4 	 1 	 17 	 18
252 	 19 	 2 	 9 	 14
234 	 5 	 3 	 6 	 13
216 	 17 	 3 	 4 	 18
204 	 18 	 3 	 4 	 17
195 	 6 	 3 	 5 	 13
90 	 13 	 3 	 5 	 6
72 	 17 	 1 	 4 	 18
68 	 18 	 1 	 4 	 17
--------------------------------------------------------


Dynamic Programming Max Weight: 34
Max Stack of Blocks:
Volume	 W	X	Y	Z
--------------------------------------------------------
2496 